In [31]:
import pickle
with open('922-Standard-SDv1.pickle', 'rb') as f:
    sdvg = pickle.load(f)
sdvg

{'0-The black chair is on the right of the wooden table': <networkx.classes.digraph.DiGraph at 0x7f325ce8fd60>,
 '1-The black chair is on the right of the wooden table': <networkx.classes.digraph.DiGraph at 0x7f325ce8ffa0>,
 '2-The black chair is on the right of the wooden table': <networkx.classes.digraph.DiGraph at 0x7f325ce90c40>,
 '3-The black chair is on the right of the wooden table': <networkx.classes.digraph.DiGraph at 0x7f325ceaf8e0>,
 '0-The round clock was mounted on the white wall': <networkx.classes.digraph.DiGraph at 0x7f325ceae950>,
 '1-The round clock was mounted on the white wall': <networkx.classes.digraph.DiGraph at 0x7f325ceae860>,
 '2-The round clock was mounted on the white wall': <networkx.classes.digraph.DiGraph at 0x7f325ceaead0>,
 '3-The round clock was mounted on the white wall': <networkx.classes.digraph.DiGraph at 0x7f325ceae4a0>,
 '0-The fluffy pillow was on the left of the striped blanket': <networkx.classes.digraph.DiGraph at 0x7f325ceaea40>,
 '1-The flu

In [32]:
import networkx as nx
class Scene_Graph:
    def __init__(self):
        self.graph = nx.MultiDiGraph()
    def add_node(self, node):
        node_idx = node
        if node in self.graph.nodes:
            for i in range(1000):
                node_idx = node + str(i)
                if node_idx not in self.graph.nodes:
                    break
                node_idx = node
            self.graph.add_node(node_idx)
            return node_idx
    def add_attribute(self, node, attribute, value):
        self.graph.nodes[node][attribute] = value
    def add_relationship(self, node1, node2, relationship):
        self.graph.add_edge(node1, node2, relationship=relationship)
    def get_nodes(self):
        return self.graph.nodes()
    def get_edges(self):
        return self.graph.edges()
    def get_attributes(self, node):
        return self.graph.nodes[node]
    def get_relationships(self, node1, node2):
        return self.graph[node1][node2][0]['relationship']

In [41]:
g1 = sdvg['2-The black chair is on the right of the wooden table']
g2 = sdvg['3-The black chair is on the right of the wooden table']

In [42]:
g1

{}

In [49]:
g1['chair']['chair1']

{'relationship': 'right$sit'}

In [9]:
g2.nodes()

NodeView(('chair', 'chair0', 'table', 'chair1', 'chair2', 'chair3', 'chair_brown', 'chair_chair', 'chair_wood', 'chair0_brown', 'chair0_chair', 'chair0_wood', 'table_brown', 'table_rectangle', 'table_wood', 'chair1_brown', 'chair1_round', 'chair1_wood', 'chair2_brown', 'chair2_round', 'chair2_wood', 'chair3_brown', 'chair3_round', 'chair3_leather'))

In [13]:
ed = nx.optimize_graph_edit_distance(g1, g2)

In [14]:
for e in ed:
    print(e)

45.0
43.0
41.0
39.0
37.0
35.0
33.0


In [15]:
nx.graph_edit_distance(g1, g2)

33.0

In [18]:
def getMCS(G_source, G_new):
    matching_graph=nx.Graph()

    for n1,n2,attr in G_new.edges(data=True):
        if G_source.has_edge(n1,n2) :
            matching_graph.add_edge(n1,n2,weight=1)

    graphs = list(nx.connected_component_subgraphs(matching_graph))

    mcs_length = 0
    mcs_graph = nx.Graph()
    for i, graph in enumerate(graphs):

        if len(graph.nodes()) > mcs_length:
            mcs_length = len(graph.nodes())
            mcs_graph = graph

    return mcs_graph

In [19]:
getMCS(g1, g2)

AttributeError: module 'networkx' has no attribute 'connected_component_subgraphs'

In [22]:
g2.nodes()

NodeView(('chair', 'chair0', 'table', 'chair1', 'chair2', 'chair3', 'chair_brown', 'chair_chair', 'chair_wood', 'chair0_brown', 'chair0_chair', 'chair0_wood', 'table_brown', 'table_rectangle', 'table_wood', 'chair1_brown', 'chair1_round', 'chair1_wood', 'chair2_brown', 'chair2_round', 'chair2_wood', 'chair3_brown', 'chair3_round', 'chair3_leather'))

In [181]:
import spacy
nlp = spacy.load('en_core_web_sm')
text = 'The black chair is on the right of the silver desk'
doc = nlp(text)

In [182]:

# doc = nlp( "spaCy uses the terms head and child to describe the words" )
for token in doc:
    print('{0}({1}) <-- {2} -- {3}({4})'.format(token.text, token.pos_, token.dep_, token.head.text, token.head.tag_))

The(DET) <-- det -- chair(NN)
black(ADJ) <-- amod -- chair(NN)
chair(NOUN) <-- nsubj -- is(VBZ)
is(AUX) <-- ROOT -- is(VBZ)
on(ADP) <-- prep -- is(VBZ)
the(DET) <-- det -- right(NN)
right(NOUN) <-- pobj -- on(IN)
of(ADP) <-- prep -- right(NN)
the(DET) <-- det -- desk(NN)
silver(NOUN) <-- compound -- desk(NN)
desk(NOUN) <-- pobj -- of(IN)


In [258]:
from collections import defaultdict

def parser_func(doc):
    parse_dict = defaultdict(dict)
    relation_dict = dict()
    for token in doc:
        print(token.tag_, token.dep_, token.text)
        if 'NN' in token.head.tag_ and token.dep_ == 'amod':
            if parse_dict[token.head.text].get('attributes'):
                parse_dict[token.head.text]['attributes'].append(token.text)
            else:
                parse_dict[token.head.text]['attributes'] = [token.text]
        elif token.pos_ == 'ADJ' and token.dep_ == 'conj':
            if token.head.pos_ == 'ADJ' and token.dep_ == 'amod':
                noun_node = token.head.head
                assert noun_node.pos_ == 'NOUN', 'conj error'
                parse_dict[noun_node.text]['attributes']
        elif 'NN' in token.tag_ and 'nsubj' in token.dep_: # verb root
            relation = token.head.lemma_
            # next_node = token.head.head
            if token.text == 'dog': print('1')
            parse_dict[token.text]['relationship'] = relation
            relation_dict[relation] = token.text
        elif token.dep_ == 'prep' and token.head.dep_ == 'ROOT' and 'NN' in token.head.tag_: #noun root and prep relation
            if token.text == 'dog':print('2')
            relation = token.lemma_
            relation_dict[relation] = token.head.text

        elif 'NN' in token.tag_ and (token.dep_ == 'dobj' or token.dep_ == 'pobj') and token.text != 'right' and token.text != 'left':
            # print(token)
            old_token = token
            flag = ''
            last_head = ''
            # print('------')
            # print(token)
            while True:
                
                if token.pos_ == 'VERB' and token.dep_ == 'ROOT':
                    break
                if token.pos_ == 'ADP' and token.dep_ == 'prep' and token.head.dep_ == 'ROOT' and 'NN' in token.head.tag_: #for dog under cat
                    break
                if token.text in ('right', 'left'):
                    flag = 'case3'
                    break
                if token.pos_ == 'ADP' and token.dep_ == 'prep' and token.head.dep_ == 'ROOT' and 'AUX' in token.head.pos_: #for dog under cat
                    flag = 'case2'
                    break
                if token.head.dep_ == 'ROOT' and 'AUX' in token.head.pos_:
                    flag = 'case1'
                    break
                # print(token, token.pos_)
                # print('******')
                # print(last_head, token.text)
                last_head = token.lemma_
                token = token.head
                # print('***2**')
                # print(last_head, token.text)
                if last_head == token.lemma_:
                    # print(last_head)
                    return parse_dict, relation_dict
            
            # print(token.text, token.head)
            if flag == 'case1':
                relation = token.head.lemma_
            elif flag == 'case2':
                relation = token.head.lemma_
            elif flag == 'case3':
                relation = 'be'
            else:
                relation = token.lemma_
            # print(relation)
            # print(relation)
            # print(relation_dict)
            subj_txt = relation_dict[relation]
            parse_dict[subj_txt]['obj'] = old_token.text
            if old_token.text not in parse_dict:
                parse_dict[old_token.text] = {}
            if flag == 'case1':
                parse_dict[subj_txt]['relationship'] = token.lemma_
            elif flag == 'case2':
                parse_dict[subj_txt]['relationship'] = token.lemma_
            elif flag == 'case3':
                parse_dict[subj_txt]['relationship'] = token.lemma_

            # while next_node.pos_ != 'NOUN':
            #     next_node = next_node.head
    return parse_dict, relation_dict

In [259]:
a, b = parser_func(doc)

DT det The
JJ amod black
NN nsubj chair
VBZ ROOT is
IN prep on
DT det the
NN pobj right
IN prep of
DT det the
NN compound silver
NN pobj desk
------
desk
******
 desk
***2**
desk of
******
desk of
***2**
of right


In [260]:
token

desk

In [261]:
doc

The black chair is on the right of the silver desk

In [262]:
b

{'be': 'chair'}

In [263]:
a

defaultdict(dict,
            {'chair': {'attributes': ['black'],
              'relationship': 'right',
              'obj': 'desk'},
             'desk': {}})

In [28]:
import pickle
with open('noun_list.pickle', 'rb') as f:
    noun_list = pickle.load(f)

In [20]:
all_prompts = []
for k, v in noun_list.items():
    all_prompts.append(k)
all_prompts

['The bitter coffee sat next to the sweet donut and the savory bagel',
 'The black camera was mounted on the silver tripod',
 'The black camera was next to the white tripod',
 'The black chair is on the right of the silver desk',
 'The black chair is on the right of the wooden table',
 'The black chair is on top of the blue rug',
 'The black chair was next to the silver lamp',
 'The black chair was next to the silver table',
 'The black chair was on the left of the brown table',
 'The black chair was on the left of the white table',
 'The black coffee mug was on top of the silver coaster',
 'The black headphones were next to the green laptop',
 'The black headphones were next to the green phone',
 'The black keyboard was on top of the white desk',
 'The black lamp was on top of the white nightstand',
 'The black mouse was next to the green mouse pad',
 'The black mouse was on the left of the red keyboard',
 'The black mug was on top of the white coaster',
 'The black pen was next to th

In [180]:
prompt2parse = dict()
from tqdm import tqdm
for prompt in tqdm(all_prompts):
    #try:
    doc = nlp(prompt)
    parse_dict = parser_func(doc)
    prompt2parse[prompt] = parse_dict
    # except:
    #     continue

print(len(all_prompts), len(list_of_dict))
prompt2parse

 12%|█▏        | 120/994 [00:00<00:02, 294.79it/s]

1
1
1
1
1
1


100%|██████████| 994/994 [00:03<00:00, 289.13it/s]


994 994


{'The bitter coffee sat next to the sweet donut and the savory bagel': (defaultdict(dict,
              {'coffee': {'attributes': ['bitter'], 'relationship': 'sit'},
               'donut': {'attributes': ['sweet']}}),
  {'sit': 'coffee'}),
 'The black camera was mounted on the silver tripod': (defaultdict(dict,
              {'camera': {'attributes': ['black'], 'relationship': 'mount'},
               'tripod': {'attributes': ['silver']}}),
  {'mount': 'camera'}),
 'The black camera was next to the white tripod': (defaultdict(dict,
              {'camera': {'attributes': ['black'], 'relationship': 'be'},
               'tripod': {'attributes': ['white']}}),
  {'be': 'camera'}),
 'The black chair is on the right of the silver desk': (defaultdict(dict,
              {'chair': {'attributes': ['black'], 'relationship': 'be'}}),
  {'be': 'chair'}),
 'The black chair is on the right of the wooden table': (defaultdict(dict,
              {'chair': {'attributes': ['black'], 'relationship': 'b

In [162]:
doc

The bold, striking patterns of the tiger's stripes blended seamlessly with the dappled light of the jungle, a creature of stealth and beauty

In [163]:

# doc = nlp( "spaCy uses the terms head and child to describe the words" )
for token in doc:
    print('{0}({1}) <-- {2} -- {3}({4})'.format(token.text, token.pos_, token.dep_, token.head.text, token.head.tag_))

The(DET) <-- det -- patterns(NNS)
bold(ADJ) <-- amod -- patterns(NNS)
,(PUNCT) <-- punct -- patterns(NNS)
striking(VERB) <-- amod -- patterns(NNS)
patterns(NOUN) <-- ROOT -- patterns(NNS)
of(ADP) <-- prep -- patterns(NNS)
the(DET) <-- det -- tiger(NN)
tiger(NOUN) <-- poss -- stripes(NNS)
's(PART) <-- case -- tiger(NN)
stripes(NOUN) <-- pobj -- of(IN)
blended(VERB) <-- acl -- stripes(NNS)
seamlessly(ADV) <-- advmod -- blended(VBN)
with(ADP) <-- prep -- blended(VBN)
the(DET) <-- det -- light(NN)
dappled(ADJ) <-- amod -- light(NN)
light(NOUN) <-- pobj -- with(IN)
of(ADP) <-- prep -- light(NN)
the(DET) <-- det -- jungle(NN)
jungle(NOUN) <-- pobj -- of(IN)
,(PUNCT) <-- punct -- patterns(NNS)
a(DET) <-- det -- creature(NN)
creature(NOUN) <-- appos -- patterns(NNS)
of(ADP) <-- prep -- creature(NN)
stealth(NOUN) <-- pobj -- of(IN)
and(CCONJ) <-- cc -- stealth(NN)
beauty(NOUN) <-- conj -- stealth(NN)


In [164]:
token

beauty

In [170]:
token.head.head.head.head.head.head.head.head.head.head.head.head.head.head.head.head.head.head.head

patterns

In [148]:
parser_func(doc)

KeyboardInterrupt: 